#That is Algorithms is developed by me

In [1]:
import numpy as np

# Data: simple sequence
data = np.linspace(0, 2*np.pi, 100)
sin_wave = np.sin(data)

# Hyperparameters
input_size = 1
hidden_size = 10
output_size = 1
lr = 0.01
epochs = 200

# Weights
Wxh = np.random.randn(hidden_size, input_size) * 0.01
Whh = np.random.randn(hidden_size, hidden_size) * 0.01
Why = np.random.randn(output_size, hidden_size) * 0.01
bh = np.zeros((hidden_size, 1))
by = np.zeros((output_size, 1))

# Training
for epoch in range(epochs):
    h_prev = np.zeros((hidden_size, 1))
    loss = 0
    # Store values for BPTT
    xs, hs, ys, targets = {}, {}, {}, {}

    for t in range(len(sin_wave)-1):
        xs[t] = np.array([[sin_wave[t]]])
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, h_prev) + bh)
        ys[t] = np.dot(Why, hs[t]) + by
        targets[t] = np.array([[sin_wave[t+1]]])
        loss += (ys[t] - targets[t])**2
        h_prev = hs[t]

    # Backpropagation Through Time
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dh_next = np.zeros_like(h_prev)

    for t in reversed(range(len(sin_wave)-1)):
        dy = ys[t] - targets[t]
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dh_next
        dh_raw = (1 - hs[t]**2) * dh
        dbh += dh_raw
        dWxh += np.dot(dh_raw, xs[t].T)
        dWhh += np.dot(dh_raw, hs[t-1].T if t != 0 else np.zeros_like(h_prev).T)
        dh_next = np.dot(Whh.T, dh_raw)

    # Update weights
    for param, dparam in zip([Wxh, Whh, Why, bh, by],
                             [dWxh, dWhh, dWhy, dbh, dby]):
        param -= lr * dparam

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.sum()/len(sin_wave):.6f}")


Epoch 0, Loss: 0.494868
Epoch 20, Loss: 0.004062
Epoch 40, Loss: 0.003583
Epoch 60, Loss: 0.004132
Epoch 80, Loss: 0.004096
Epoch 100, Loss: 0.002780
Epoch 120, Loss: 0.002325
Epoch 140, Loss: 0.002157
Epoch 160, Loss: 0.002033
Epoch 180, Loss: 0.001927
